### Importing Python Libraries

In [10]:
import pandas as pd
import sagemaker.amazon.common as smac
from sklearn.model_selection import train_test_split
import boto3
import sagemaker
from io import StringIO
import joblib
import statsmodels.api as sm
from typing import List, Union
from dotenv import load_dotenv
import os

In [20]:
load_dotenv()

True

In [ ]:
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

bucket = sagemaker.Session().default_bucket()

### Data Source

In [ ]:
s3 = boto3.client("s3")

filename = os.getenv("CSV_FILENAME")
s3.download_file(os.getenv('S3_BUKCET'), os.getenv('S3_BUCKET_PREFIX'), filename)
data = pd.read_csv(filename)

### Exploratory Data Analysis

In [52]:
# columns
data.columns

Index(['Security_ID', 'BDC_Name', 'FV_Percentage_Cost', 'FV_Per_Par',
       'Non_Accrual', 'Security_Type', 'Non_Qualifying', 'Loan_Tenure',
       'FV_Percentage_Cost_Q1', 'FV_Per_Par_Q1', 'Non_Accrual_Q1',
       'Security_Type_Q1', 'Non_Qualifying_Q1', 'Loan_Tenure_Q1',
       'FV_Percentage_Cost_Q2', 'FV_Per_Par_Q2', 'Non_Accrual_Q2',
       'Security_Type_Q2', 'Non_Qualifying_Q2', 'Loan_Tenure_Q2'],
      dtype='object')

In [53]:
# Shape of the data
print(data.shape)

(13720, 20)


In [54]:
# Data Types
data.dtypes

Security_ID               object
BDC_Name                  object
FV_Percentage_Cost       float64
FV_Per_Par               float64
Non_Accrual                int64
Security_Type             object
Non_Qualifying             int64
Loan_Tenure              float64
FV_Percentage_Cost_Q1    float64
FV_Per_Par_Q1            float64
Non_Accrual_Q1             int64
Security_Type_Q1          object
Non_Qualifying_Q1          int64
Loan_Tenure_Q1           float64
FV_Percentage_Cost_Q2    float64
FV_Per_Par_Q2            float64
Non_Accrual_Q2             int64
Security_Type_Q2          object
Non_Qualifying_Q2          int64
Loan_Tenure_Q2           float64
dtype: object

In [55]:
# show top 5 rows
data.head(5)

,Security_ID,BDC_Name,FV_Percentage_Cost,FV_Per_Par,Non_Accrual,Security_Type,Non_Qualifying,Loan_Tenure,FV_Percentage_Cost_Q1,FV_Per_Par_Q1,Non_Accrual_Q1,Security_Type_Q1,Non_Qualifying_Q1,Loan_Tenure_Q1,FV_Percentage_Cost_Q2,FV_Per_Par_Q2,Non_Accrual_Q2,Security_Type_Q2,Non_Qualifying_Q2,Loan_Tenure_Q2
0,'5d564e22bc924,"Venture Lending & Leasing IX, Inc.",100.000,97.955,0,First Lien Senior Secured,0,-20.0,100.000,97.675,0,First Lien Senior Secured,0,-20.0,100.00,97.405,0,First Lien Senior Secured,0,-20.0
1,'591d34a7da730,"Golub Capital BDC, Inc",98.100,97.990,0,First Lien Senior Secured,0,-16.0,86.145,85.000,1,First Lien Senior Secured,0,-52.0,84.96,84.995,1,First Lien Senior Secured,0,-52.0
2,'5aafd3aae0512,Newtek Business Services Corp.,NaN,97.650,0,First Lien Senior Secured,0,27.0,NaN,97.700,0,First Lien Senior Secured,0,27.0,NaN,95.560,0,First Lien Senior Secured,0,27.0
3,'5d42963a6bac0,"Sixth Street Specialty Lending, Inc.",107.665,112.350,0,First Lien Senior Secured,0,-4.0,109.850,114.535,0,First Lien Senior Secured,0,-4.0,102.83,106.335,0,First Lien Senior Secured,0,-4.0
4,'591ade555362b,"Bain Capital Specialty Finance, Inc",86.070,86.000,0,First Lien Senior Secured,0,-23.0,82.640,82.500,0,First Lien Senior Secured,0,-23.0,79.44,79.250,0,First Lien Senior Secured,0,-23.0


In [56]:
# data objects
data.describe()

,FV_Percentage_Cost,FV_Per_Par,Non_Accrual,Non_Qualifying,Loan_Tenure,FV_Percentage_Cost_Q1,FV_Per_Par_Q1,Non_Accrual_Q1,Non_Qualifying_Q1,Loan_Tenure_Q1,FV_Percentage_Cost_Q2,FV_Per_Par_Q2,Non_Accrual_Q2,Non_Qualifying_Q2,Loan_Tenure_Q2
count,9729.000000,10206.00000,13720.000000,13720.000000,11202.000000,9218.000000,9958.000000,13720.000000,13720.000000,10921.000000,8910.000000,9648.000000,13720.000000,13720.000000,10418.000000
mean,98.321914,95.95886,0.028863,0.070918,11.514084,97.937096,95.416602,0.030102,0.063848,7.370660,97.537538,94.108483,0.034913,0.063557,3.978200
std,23.403210,16.10237,0.167427,0.256698,55.981540,38.003815,16.290751,0.170874,0.244492,56.282451,163.928576,16.757192,0.183565,0.243971,57.299719
min,0.220000,0.10000,0.000000,0.000000,-128.000000,0.150000,0.100000,0.000000,0.000000,-128.000000,0.020000,0.010000,0.000000,0.000000,-128.000000
25%,98.740000,96.96250,0.000000,0.000000,-19.000000,97.552500,95.900000,0.000000,0.000000,-24.000000,95.150000,93.685000,0.000000,0.000000,-28.000000
50%,100.000000,99.50000,0.000000,0.000000,0.000000,100.000000,99.010000,0.000000,0.000000,-4.000000,99.740000,98.390000,0.000000,0.000000,-8.000000
75%,101.760000,100.00000,0.000000,0.000000,18.000000,101.529167,100.000000,0.000000,0.000000,14.000000,100.850000,100.000000,0.000000,0.000000,10.000000
max,1914.280000,242.51000,1.000000,1.000000,349.000000,3396.220000,320.100000,1.000000,1.000000,349.000000,15487.020000,225.000000,1.000000,1.000000,349.000000


### Data Preprocessing

In [57]:
# Drop rows where ANY value is NaN
data = data.dropna()

In [58]:
# Loan Tenures must be >= 1
data = data[data['Loan_Tenure'] >= 1]
data = data[data['Loan_Tenure_Q1'] >= 1]
data = data[data['Loan_Tenure_Q2'] >= 1]

In [59]:
# FV % Cost Normalization
data['fv_percentage_cost_normalized'] = data['FV_Percentage_Cost'].apply(
        lambda x: 10 if x >= 100 
                  else 8 if x >= 80 
                  else 5 if x >= 70 
                  else 3 if x >= 50 
                  else 1
    )

data['fv_percentage_cost_q1_normalized'] = data['FV_Percentage_Cost_Q1'].apply(
        lambda x: 10 if x >= 100 
                  else 8 if x >= 80 
                  else 5 if x >= 70 
                  else 3 if x >= 50 
                  else 1
    )

data['fv_percentage_cost_q2_normalized'] = data['FV_Percentage_Cost_Q2'].apply(
        lambda x: 10 if x >= 100 
                  else 8 if x >= 80 
                  else 5 if x >= 70 
                  else 3 if x >= 50 
                  else 1
    )

In [60]:
# FV / Par Normalization
data['fv_par_normalized'] = data['FV_Per_Par'].apply(
        lambda x: 10 if x >= 100 
                  else 8 if x >= 80 
                  else 5 if x >= 70 
                  else 3 if x >= 50 
                  else 1
    )

data['fv_par_q1_normalized'] = data['FV_Per_Par_Q1'].apply(
        lambda x: 10 if x >= 100 
                  else 8 if x >= 80 
                  else 5 if x >= 70 
                  else 3 if x >= 50 
                  else 1
    )

data['fv_par_q2_normalized'] = data['FV_Per_Par_Q2'].apply(
        lambda x: 10 if x >= 100 
                  else 8 if x >= 80 
                  else 5 if x >= 70 
                  else 3 if x >= 50 
                  else 1
    )

In [61]:
# Calculate Asset Score of Latest Quarter
data['Asset_Score'] = ((data['fv_percentage_cost_normalized']) * (1.0/3))  + (data['fv_par_normalized'] * (1.0/3))  + (data['Non_Accrual'] * (1.0/3))

In [62]:
# Seniority Normalization
data['seniority_q1_normalized'] = data['Security_Type_Q1'].apply(
       lambda x: 6 if x == 'First Lien Senior Secured'
            else 5 if x == 'Second Lien Senior Secured'
            else 4 if x == 'Subordinated'
            else 3
)

data['seniority_q2_normalized'] = data['Security_Type_Q2'].apply(
       lambda x: 6 if x == 'First Lien Senior Secured'
            else 5 if x == 'Second Lien Senior Secured'
            else 4 if x == 'Subordinated'
            else 3
)

In [63]:
data.describe()

,FV_Percentage_Cost,FV_Per_Par,Non_Accrual,Non_Qualifying,Loan_Tenure,FV_Percentage_Cost_Q1,FV_Per_Par_Q1,Non_Accrual_Q1,Non_Qualifying_Q1,Loan_Tenure_Q1,...,Loan_Tenure_Q2,fv_percentage_cost_normalized,fv_percentage_cost_q1_normalized,fv_percentage_cost_q2_normalized,fv_par_normalized,fv_par_q1_normalized,fv_par_q2_normalized,Asset_Score,seniority_q1_normalized,seniority_q2_normalized
count,2905.000000,2905.000000,2905.000000,2905.000000,2905.000000,2905.000000,2905.000000,2905.000000,2905.000000,2905.000000,...,2905.000000,2905.000000,2905.000000,2905.000000,2905.000000,2905.000000,2905.000000,2905.000000,2905.00000,2905.000000
mean,99.124937,98.101596,0.040964,0.086403,61.098926,98.517673,97.396679,0.040275,0.085026,60.872822,...,60.805610,8.954905,8.816523,8.511532,8.685714,8.569363,8.265060,5.893861,5.77969,5.780034
std,16.163859,16.878776,0.198241,0.281006,74.538475,16.255551,16.865824,0.196638,0.278968,74.525200,...,74.402974,1.894523,1.900704,2.015596,1.921322,1.924386,2.013377,1.173606,0.62075,0.619484
min,0.990000,0.990000,0.000000,0.000000,1.000000,1.090000,1.000000,0.000000,0.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.666667,3.00000,3.000000
25%,97.325000,96.750000,0.000000,0.000000,9.000000,95.900000,95.000000,0.000000,0.000000,9.000000,...,9.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,5.333333,6.00000,6.000000
50%,100.890000,100.000000,0.000000,0.000000,27.000000,100.490000,99.490000,0.000000,0.000000,27.000000,...,27.000000,10.000000,10.000000,8.000000,10.000000,8.000000,8.000000,6.666667,6.00000,6.000000
75%,105.230000,104.440000,0.000000,0.000000,61.000000,104.955000,103.840000,0.000000,0.000000,61.000000,...,61.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,6.666667,6.00000,6.000000
max,210.620000,210.620000,1.000000,1.000000,349.000000,202.670000,202.670000,1.000000,1.000000,349.000000,...,349.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,7.000000,6.00000,6.000000


In [64]:
# Drop Unnecessary Columns
data = data.drop(['FV_Per_Par', 'FV_Per_Par_Q1', 'FV_Per_Par_Q2', 'fv_par_normalized',
                  'FV_Percentage_Cost', 'FV_Percentage_Cost_Q1', 'FV_Percentage_Cost_Q2', 
                  'fv_percentage_cost_normalized', 'Loan_Tenure', 'Non_Accrual', 'Non_Qualifying',
                 'BDC_Name', 'Security_ID', 'Security_Type', 'Security_Type_Q1', 'Security_Type_Q2'], axis=1)

In [65]:
data.head(10)

,Non_Accrual_Q1,Non_Qualifying_Q1,Loan_Tenure_Q1,Non_Accrual_Q2,Non_Qualifying_Q2,Loan_Tenure_Q2,fv_percentage_cost_q1_normalized,fv_percentage_cost_q2_normalized,fv_par_q1_normalized,fv_par_q2_normalized,Asset_Score,seniority_q1_normalized,seniority_q2_normalized
8,0,0,24.0,0,0,24.0,8,8,8,8,6.666667,6,6
11,0,0,51.0,0,0,51.0,8,5,8,5,5.333333,6,6
12,0,0,194.0,0,0,194.0,10,10,10,10,6.666667,6,6
15,0,1,4.0,0,1,4.0,1,1,1,1,0.666667,3,3
16,0,0,67.0,0,0,67.0,8,8,3,3,3.666667,3,3
17,0,0,13.0,0,0,13.0,10,10,10,8,6.666667,6,6
20,0,1,12.0,0,1,12.0,8,8,8,8,5.333333,5,5
21,0,0,51.0,0,0,51.0,8,5,8,5,5.333333,6,6
23,0,0,26.0,0,0,26.0,10,10,10,10,6.666667,6,6
28,0,0,12.0,0,0,12.0,10,8,8,8,5.333333,6,6


In [66]:
FEATURE_COLUMNS = [
    "Non_Accrual_Q1",
    # "Non_Qualifying_Q1",
    # "Loan_Tenure_Q1",
    "Non_Accrual_Q2",
    # "Non_Qualifying_Q2",
    # "Loan_Tenure_Q2",
    "fv_percentage_cost_q1_normalized",
    "fv_percentage_cost_q2_normalized",
    "fv_par_q1_normalized",
    "fv_par_q2_normalized",
    "seniority_q1_normalized",
    # "seniority_q2_normalized"
]
X = data[FEATURE_COLUMNS]

In [67]:
TARGET_COLUMN = "Asset_Score"
y = data[TARGET_COLUMN]

In [68]:
# Split Training & Testing Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Training Data CSV and Upload to S3

bucket_name = os.getenv("S3_BUKCET")
train_key = os.getenv("TRAIN_CSV_PATH_KEY")

train_buffer = StringIO()
train_data = pd.concat([X_train, y_train], axis=1)
train_data.to_csv(train_buffer, index=False)
s3 = boto3.resource('s3')

# Upload to S3
s3.Object(bucket_name, train_key).put(Body=train_buffer.getvalue())


{'ResponseMetadata': {'RequestId': 'FDGRH2T6ED342ER8',
  'HostId': 'iWAHFMaTm1uHEIxDzjoi385aN0rN1ABjdxHY06Ogb8UlF017b65QgxnXAYOvNEMurhF3zAYTaB/WkhTOxVnc3xGpwWxWOFYNu+UPcnlFOqI=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'iWAHFMaTm1uHEIxDzjoi385aN0rN1ABjdxHY06Ogb8UlF017b65QgxnXAYOvNEMurhF3zAYTaB/WkhTOxVnc3xGpwWxWOFYNu+UPcnlFOqI=',
   'x-amz-request-id': 'FDGRH2T6ED342ER8',
   'date': 'Wed, 10 Sep 2025 12:55:57 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"46edcd2b609c3c5abf9eb7aa21741cee"',
   'x-amz-checksum-crc32': 'R3fRYA==',
   'x-amz-checksum-type': 'FULL_OBJECT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"46edcd2b609c3c5abf9eb7aa21741cee"',
 'ChecksumCRC32': 'R3fRYA==',
 'ChecksumType': 'FULL_OBJECT',
 'ServerSideEncryption': 'AES256'}

In [ ]:
# Testing Data CSV and Upload to S3

test_key = os.getenv("TEST_KEY_PATH")

test_buffer = StringIO()
test_data = pd.concat([X_test, y_test], axis=1)
test_data.to_csv(test_buffer, index=False)
s3 = boto3.resource('s3')

# Upload to S3
s3.Object(bucket_name, test_key).put(Body=test_buffer.getvalue())


{'ResponseMetadata': {'RequestId': 'FDGJ1073QZ04MNZ3',
  'HostId': 'nBpuYOE+nxB7dymUc+SfZS6GSSZAcCdJHSW9Q0rf5EN5mGIO4HDce9piXRkhGUNucYv/L4kc4Ow=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'nBpuYOE+nxB7dymUc+SfZS6GSSZAcCdJHSW9Q0rf5EN5mGIO4HDce9piXRkhGUNucYv/L4kc4Ow=',
   'x-amz-request-id': 'FDGJ1073QZ04MNZ3',
   'date': 'Wed, 10 Sep 2025 12:55:57 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"905dddf081da27cdada98f74af50654e"',
   'x-amz-checksum-crc32': 'YlJ49A==',
   'x-amz-checksum-type': 'FULL_OBJECT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"905dddf081da27cdada98f74af50654e"',
 'ChecksumCRC32': 'YlJ49A==',
 'ChecksumType': 'FULL_OBJECT',
 'ServerSideEncryption': 'AES256'}

In [ ]:
X_with_const = sm.add_constant(X)
model = sm.OLS(y, X_with_const).fit()
joblib.dump(model, 'Regression_model')
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            Asset_Score   R-squared:                       0.854
Model:                            OLS   Adj. R-squared:                  0.853
Method:                 Least Squares   F-statistic:                     2414.
Date:                Wed, 10 Sep 2025   Prob (F-statistic):               0.00
Time:                        12:55:56   Log-Likelihood:                -1795.1
No. Observations:                2905   AIC:                             3606.
Df Residuals:                    2897   BIC:                             3654.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const   

In [ ]:
# save model to S3
s3_client = boto3.client('s3')

s3_key = os.getenv("MODEL_KEY_PATH")

try:
    s3_client.upload_file('Regression_model', bucket_name, s3_key)
    print(f"Model '{'Regression_model'}' successfully uploaded to s3://{bucket_name}/{s3_key}")
except Exception as e:
    print(f"Error uploading model to S3: {e}")

Model 'Regression_model' successfully uploaded to s3://sagemaker-us-west-2-109675754062/jupyter_notebook/model/Regression_model.pkl
